<a href="https://colab.research.google.com/github/sheyhusna/Computer-Security-Cryptography-Design-Application-Assignment/blob/main/Tornado_25_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile Tornado-25.cpp
#include <iostream>
#include <vector>
#include <string>
#include <random>

using namespace std;

// Base64 encoding table
const string base64_chars =
    "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    "abcdefghijklmnopqrstuvwxyz"
    "0123456789+/";

// Rotate left
unsigned char rotateLeft(unsigned char byte, int shift = 1) {
    return (byte << shift) | (byte >> (8 - shift));
}

// Rotate right
unsigned char rotateRight(unsigned char byte, int shift = 1) {
    return (byte >> shift) | (byte << (8 - shift));
}

// Generate rotated keystream
vector<unsigned char> generateKeystream(int seed, int length) {
    mt19937 rng(seed);
    uniform_int_distribution<int> dist(0, 255);
    vector<unsigned char> keystream;

    for (int i = 0; i < length; ++i) {
        unsigned char k = dist(rng);
        if (k % 2 == 0) {
            k = rotateLeft(k);
        } else {
            k = rotateRight(k);
        }
        keystream.push_back(k);
    }

    return keystream;
}

// Encrypt
string encrypt(const string& plaintext, int key) {
    vector<unsigned char> keystream = generateKeystream(key, plaintext.length());
    string encrypted = "";

    for (int i = 0; i < plaintext.length(); ++i) {
        char c = plaintext[i] ^ keystream[i];
        encrypted += c;
    }

    return encrypted;
}

// Decrypt
string decrypt(const string& ciphertext, int key) {
    vector<unsigned char> keystream = generateKeystream(key, ciphertext.length());
    string decrypted = "";

    for (int i = 0; i < ciphertext.length(); ++i) {
        char p = ciphertext[i] ^ keystream[i];
        decrypted += p;
    }

    return decrypted;
}

// Base64 encode
string base64Encode(const string& data) {
    string encoded;
    int val = 0, valb = -6;
    for (unsigned char c : data) {
        val = (val << 8) + c;
        valb += 8;
        while (valb >= 0) {
            encoded.push_back(base64_chars[(val >> valb) & 0x3F]);
            valb -= 6;
        }
    }
    if (valb > -6) encoded.push_back(base64_chars[((val << 8) >> (valb + 8)) & 0x3F]);
    while (encoded.size() % 4) encoded.push_back('=');
    return encoded;
}

// Base64 decode
string base64Decode(const string& encoded) {
    string decoded;
    vector<int> T(256, -1);
    for (int i = 0; i < 64; i++) T[base64_chars[i]] = i;

    int val = 0, valb = -8;
    for (unsigned char c : encoded) {
        if (T[c] == -1) break;
        val = (val << 6) + T[c];
        valb += 6;
        if (valb >= 0) {
            decoded.push_back(char((val >> valb) & 0xFF));
            valb -= 8;
        }
    }
    return decoded;
}

int main() {
    string message = "THE MANAGER IS HERE";
    int key = 67943;

    // Encrypt and encode
    string encrypted = encrypt(message, key);
    string encoded = base64Encode(encrypted);

    // Decode and decrypt
    string decoded = base64Decode(encoded);
    string decrypted = decrypt(decoded, key);

    // Output
    cout << "Original Message: " << message << endl;
    cout << "Encrypted (Base64): " << encoded << endl;
    cout << "Decrypted Message: " << decrypted << endl;

    return 0;
}

Writing Tornado-25.cpp


In [2]:
!g++ Tornado-25.cpp -o Tornado-25

In [4]:
!./Tornado-25

Original Message: THE MANAGER IS HERE
Encrypted (Base64): ks3ZyuWlRpu7AK6sjYZMHEjjFQ==
Decrypted Message: THE MANAGER IS HERE
